# Matching Model
To find the expected effect of the intervention on the population, we match each treated individual with one or more untreated individuals which are "almost the same" as him or her.

In [1]:
%matplotlib inline
from causallib.datasets import load_nhefs
from causallib.estimation import IPW,Matcher
from causallib.evaluation import PropensityEvaluator
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

#### Data:
The effect of quitting to smoke on weight loss.  
Data example is taken from [Hernan and Robins Causal Inference Book](https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/)

In [167]:
augmented_data = load_nhefs()
augmented_data.X.join(augmented_data.a).join(augmented_data.y).head()

,age,race,sex,smokeintensity,smokeyrs,wt71,active_1,active_2,education_2,education_3,education_4,education_5,exercise_1,exercise_2,age^2,wt71^2,smokeintensity^2,smokeyrs^2,qsmk,wt82_71
0,42,1,0,30,29,79.04,0,0,0,0,0,0,0,1,1764,6247.3216,900,841,0,-10.093960
1,36,0,0,20,24,58.63,0,0,1,0,0,0,0,0,1296,3437.4769,400,576,0,2.604970
2,56,1,1,20,26,56.81,0,0,1,0,0,0,0,1,3136,3227.3761,400,676,0,9.414486
3,68,1,0,3,53,59.42,1,0,0,0,0,0,0,1,4624,3530.7364,9,2809,0,4.990117
4,40,0,0,20,19,87.09,1,0,1,0,0,0,1,0,1600,7584.6681,400,361,0,4.989251


When we are looking for nearby data points to match against, the one hot encoding may not be the best choice. Augmented features are also not needed and may introduce bias. So instead of one hot encoding the categorical variables, we binarize them to "high/low" values and we do not augment the continuous variables.

In [168]:
def binarize(df,column_name):
    df = df.copy()
    m=df[column_name].median()
    balance = lambda i : np.abs(0.5 - (df[column_name] < i).sum()/len(df))
    mstar = min([m-1,m,m+1],key=balance)
    df = df.assign(**{column_name:(df[column_name] < mstar).astype(int)})
    df = df.rename(columns={column_name: column_name + f"<{mstar}"})
    return df

def get_matching_data():
    data = load_nhefs(onehot=False,augment=False)
    data.X = binarize(data.X,"education")
    data.X = binarize(data.X,"exercise")
    data.X = binarize(data.X,"active")
    return data
binarized_data = get_matching_data()
X,a,y = binarized_data.X,binarized_data.a,binarized_data.y

We still want the propensity to use the augmented data, though, so we will pretrain a propensity vector that is indexed like the rest of the data and use it when necessary.

In [169]:
learner = LogisticRegression(solver="liblinear")
learner.fit(augmented_data.X,augmented_data.a)
propensity = learner.predict_proba(augmented_data.X)[:,0]
propensity_df = augmented_data.X.assign(propensity=propensity)
propensity_only_df = augmented_data.X.assign(propensity=propensity).pop("propensity")
class PropensityTransformer:
    def __init__(self):
        self.include_covariates=False
    def fit(self,X,p):
        self.X = X.assign(propensity=p)
    def transform(self,X):
        if self.include_covariates:
            return self.X.loc[X.index]["propensity"]
        else:
            return self.X.loc[X.index]
propensity_transform = PropensityTransformer()
propensity_transform.fit(augmented_data.X,propensity)
propensity_transform.include_covariates =True

Currently the `Matcher` class has two functional initialization parameters: `use_propensity` and `use_covariates`. We explore the differing outcomes from using the 4 possible values of these boolean parameters.

In [184]:
matcher = Matcher()
matcher.propensity_transform = propensity_transform
matcher.caliper = 0.001
matcher.fit(binarized_data.X,binarized_data.a,binarized_data.y)
Y = matcher.estimate_individual_outcome( binarized_data.X, binarized_data.a,)
match_df_with_rep = matcher.match(X,a,y)
(Y[1] - Y[0]).mean()

3.7826753981208054

In [100]:
ipw = IPW(LogisticRegression(solver="liblinear"))
ipw.fit(augmented_data.X,augmented_data.a)
Yipw = ipw.estimate_population_outcome(augmented_data.X,augmented_data.a,augmented_data.y)
ipw_estimate = Yipw[1] - Yipw[0]

In [101]:
naive_estimate = y[a==1].mean() - y[a==0].mean()

In [ ]:
caliper = np.logspace(-5,0,20)
def check_caliper(c,with_replacement=True):
    matcher = Matcher()
    matcher.propensity_transform = propensity_transform
    matcher.caliper = c
    matcher.with_replacement = with_replacement
    matcher.fit(binarized_data.X,binarized_data.a,binarized_data.y)
    Y = matcher.estimate_individual_outcome( binarized_data.X, binarized_data.a,)
    Y = Y.dropna()
    p = len(Y) / len(matcher.covariates)
    return p, (Y[1] - Y[0]).mean()
p_with,ATE_with = zip(*[check_caliper(c,with_replacement=True) for c in caliper])
p_without,ATE_without = zip(*[check_caliper(c,with_replacement=False) for c in caliper])

In [ ]:
import seaborn as sb
with sb.axes_style("dark") as s:
    plt.semilogx(caliper,p_with,"blue")
    plt.semilogx(caliper,p_without,"purple")
    
    plt.ylabel("fraction of data matched ",color="blue")
    plt.twinx()
    plt.semilogx(caliper,ATE_with,"green",label="matching (with replacement)")
    plt.semilogx(caliper,ATE_without,"orange",label="matching (no replacement)")
    plt.ylabel("ATE",color="green")
    plt.hlines(xmin=caliper.min(),xmax=caliper.max(),y = ipw_estimate, ls="--",color="green",label="ipw")
    plt.hlines(xmin=caliper.min(),xmax=caliper.max(),y = naive_estimate, ls=":",color="green",label="naive")
    plt.legend(loc=4)
    plt.xlabel("caliper")

In [293]:
matcher = Matcher()
matcher.with_replacement = True
matcher.propensity_transform = propensity_transform
matcher.fit(X,a,y)
match_df = matcher.match(X,a,y)
ATE_with_replacement = matcher.estimate_population_outcome(X,a).diff()[1]

In [ ]:
matcher = Matcher()
matcher.with_replacement = False
matcher.propensity_transform = propensity_transform
matcher.caliper = 0.001
matcher.fit(X,a,y)
match_df = matcher.match(X,a,y)
ATE_without_replacement = matcher.estimate_population_outcome(X,a).diff()[1]

In [382]:
print(f"With replacement we find:\n{ATE_with_replacement}\nWithout replacement we find:\n{ATE_without_replacement}")

With replacement we find:
3.841734485172423
Without replacement we find:
1.650148924838708


As we see, the average differences between the treated and untreated are very low after matching, as expected. Now we turn to the average effect of treatment on the treated:

In [ ]:
att = dict((name,matcher.get_att(binarized_data.y)) for name,matcher in matchers.items())
pd.Series(att)

In [ ]:
from sklearn.covariance import MinCovDet,EmpiricalCovariance,ShrunkCovariance,GraphicalLassoCV
mcd = MinCovDet()
mcd.fit(binarized_data.X)
ec = EmpiricalCovariance()
ec.fit(binarized_data.X)
sc = ShrunkCovariance()
sc.fit(binarized_data.X)
gl = GraphicalLassoCV()
gl.fit(binarized_data.X)

In [ ]:
f,a=plt.subplots(1,4)
for ax,cov in zip(a.flatten(),[mcd,ec,sc,gl]):
    #print(cov.precision_ @ cov.covariance_)
    ax.imshow(cov.precision_ @ cov.covariance_)

In [ ]:
match = binarized_data.a.copy()
match.name = "match"

match.loc[[m.source for m in matchers["both"].matches]] = [m.targets for m in matchers["both"].matches]

In [ ]:
from sklearn.tree import DecisionTreeClassifier,plot_tree
dtc = DecisionTreeClassifier(max_depth=5)
dtc.fit(binarized_data.X, matchers["both"].weights > 0)
f,ax=plt.subplots(figsize=(42,15))
_ = plot_tree(ax=ax,decision_tree=dtc,filled=True,feature_names=matchers["both"].covariates.columns,class_names=["unmatched","matched"],fontsize=12)


In [ ]:
dtc.score(binarized_data.X, matchers["both"].weights > 0)
#roc_auc_score(matchers["both"].weights.values,dtc.predict(binarized_data.X))

matchers["both"].weights.unique()

## Next steps
Intuitively the whole method only makes sense if the treated and control are actually "near" in some meaningful way. In particular, we want to know what we can generalize from the data we have seen to new data. Maybe dimensional reduction and visualization can help?

In [ ]:
from sklearn.decomposition import PCA
pca1,pca2 = PCA(n_components=2).fit_transform(data.X).T
import seaborn as sb


In [ ]:
from matplotlib import pyplot as plt
f,a=plt.subplots(figsize=(12,12))
sb.scatterplot(x="pca1",y="pca2",hue=data.a,data=data.X.assign(pca1=pca1,pca2=pca2),ax=a)

In [ ]:
from sklearn.manifold import TSNE
tfit = TSNE().fit_transform(data.X)

In [ ]:
from matplotlib import pyplot as plt

paired =  data.a.copy(deep=True)
for i in pairs:
    paired.loc[i[1]]=-1

def matching_as_weight(X,a,control_neighbors_of_treated):
    weights =  data.a.copy(deep=True)
    for i in control_neighbors_of_treated:
        paired.loc[control.iloc[i].index] = -1

f,a=plt.subplots(figsize=(10,10),subplot_kw=dict(facecolor=".2"))
sb.scatterplot(x=tfit.T[0],y=tfit.T[1],hue=paired,palette="vlag",alpha=0.8,ax=a)
plt.grid("off")
plt.show()

In [ ]:
import numpy as np

feature_weight = np.ones(data.X.shape[1])
X_t = data.X[data.a==1]
X_c = data.X[data.a==0]
unit_weight_t = np.random.random(X_t.shape[0])
unit_weight_c = np.random.random(X_c.shape[0])
def loss(w_t,X_t,w_c,X_c,feature_weights=1):
    if feature_weights==1:
        w_f = np.ones(X_t.shape[1])
    ((w_t @ X_t)/w_t.sum() - (w_c @ X_c)/w_c.sum())**2 @ w_f
    
loss = lambda w_f: lambda w_t,w_c : 

In [ ]:
loss(feature_weight)(unit_weight_t,unit_weight_c)

In [ ]:
pw_t = propensity[(data.a==1).values]
pw_c = propensity[(data.a==0).values]
loss(feature_weight)(pw_t,pw_c)

In [61]:
from sklearn.utils import Bunch
import requests
import zipfile
from io import BytesIO,TextIOWrapper

def get_card_krueger_data(select_covariates=["EMPTOT","WAGE_ST","INCTIME","BK","KFC","ROYS","WENDYS"],outcome="EMPTOT2",dropna=True,return_column_descriptions=False):
    
    r=requests.get("https://davidcard.berkeley.edu/data_sets/njmin.zip")
    z = zipfile.ZipFile(BytesIO(r.content))
    df = pd.read_csv(z.open("public.dat"),engine="python",sep="\s+",header=None).applymap(lambda x: pd.to_numeric(x,errors="coerce"))
    
    ## Load column names and descriptions from `codebook`
    codebook = [repr(line) for line in TextIOWrapper(z.open("codebook"),"cp437")]
    #part of the codebook is not relevant
    codes = codebook[7:11]  + codebook[13:19] + codebook[21:38] + codebook[40:59]
    cols = [i.strip("'\" ").split()[0] for i in codes]
    descriptions = [" ".join(i.strip("'\" ").split()[4:]).rstrip('\\n') for i in codes]
    column_descriptions = dict(zip(cols,descriptions))
    df.columns = cols

    ## Calculate total employment following Card and Krueger's method
    df = df.assign(EMPTOT = df.EMPFT + 0.5 * df.EMPPT + df.NMGRS)
    df = df.assign(EMPTOT2 = df.EMPFT2 + 0.5 * df.EMPPT2 + df.NMGRS2)
    
    df = pd.get_dummies(df,prefix="name",columns=["CHAIN"]).rename(columns=dict(name_1="BK",name_2="KFC",name_3="ROYS",name_4="WENDYS"))
    if dropna:
        df = df.dropna(subset=select_covariates + ["STATE", outcome])
    
    ## Divide data into pre-intervention and post-intervention variables
    x = df[[c for c in df.columns if not "2" in c]]
    
    y = df[[c for c in df.columns if "2" in c]]
    y = df.pop(outcome)
    
    a = df.pop("STATE")
        
    if select_covariates is not None:
        x = x[select_covariates]
    return Bunch(x=x,a=a,y=y,descriptors=pd.Series(column_descriptions))
    

cardkrueger = get_card_krueger_data()


In [14]:
class PropensityTransformer:
    def __init__(self,include_covariates=False):
        self.include_covariates=include_covariates   
        self.learner = LogisticRegression(solver="liblinear")
    def fit(self,X,a=None,p=None):
        if p is None:
            if a is None:
                raise ValueError("Either p or a needs to be supplied")
            self.learner.fit(X,a)
            p = self.learner.predict_proba(X)[:,0]
        self.X = X.assign(propensity=p)
    def transform(self,X):
        if self.include_covariates:
            return self.X.loc[X.index]["propensity"]
        else:
            return self.X.loc[X.index]

In [66]:
from causallib.datasets import load_card_krueger_data
cardkrueger = load_card_krueger_data()

In [25]:
x,a,y=cardkrueger.x,cardkrueger.a,cardkrueger.y
match_ck=Matcher()
propensity_transform = PropensityTransformer(False)
propensity_transform.fit(x,a)
match_ck.propensity_transform = propensity_transform


In [28]:
match_ck.fit(x,a,y)
match_ck.with_replacement = True
match_ck.n_neighbors = 1
match_ck.estimate_population_outcome(x,a).diff()[1]

0.566176470588232

In [33]:
ipw_ck = IPW(LogisticRegression(solver="liblinear"))
ipw_ck.fit(x,a)
ipw_ck.estimate_population_outcome(x,a,y).diff()[1]

1.037467023643547

In [27]:
n=10
x_=pd.DataFrame(np.hstack([np.arange(n),np.arange(n)]))
a_=pd.Series(np.hstack([np.zeros(n),np.ones(n)]))
y_=pd.Series(np.hstack([np.random.rand(n),2*np.random.rand(n)]))
data_serial_x =  Bunch(x=x_,a=a_,y=y_)
test_matcher = Matcher()
test_matcher.fit(x_,a_,y_)
test_match_df = test_matcher.match(x_,a_)


In [1]:

    def bias_correcting_estimated_outcome(self, s, t, covariates):
        """Bias correcting estimators for outcome following matching

        Args:
            s (integer): Treatment value to match to (usually treatment value for control)
            t (integer): Treatment value to match from (usually treatment value for treated)

        Returns:
            tuple(pd.Series): Three different regression corrected estimators based on the
            Difference regression, Control regression and Pooled regression methods.
        """

        covmatch = self.get_covariates_of_matches(s, t, covariates)

        # Difference Regression - regress (Y1 - Y0) ~ (X1 - X0)
        y_d = covmatch.outcomes[s] - covmatch.outcomes[t]
        x_d = covmatch.delta
        fit_d = sm.OLS(y_d, sm.add_constant(x_d)).fit()
        params_d = fit_d.params
        yhat_d = covmatch.outcomes[t] + covmatch.delta @ params_d[1:]
        tau_d = (covmatch.outcomes[s] - yhat_d).mean()

        # Control Regression - regress Y0 ~ X0

        x_c = covmatch[t].set_index("sample_id")
        y_c = covmatch.outcomes[t]
        y_c.index = x_c.index

        fit_c = sm.OLS(y_c, sm.add_constant(x_c)).fit()
        params_c = fit_c.params
        yhat_c = covmatch.outcomes[t] - covmatch.delta @ params_c[1:]
        tau_c = (covmatch.outcomes[s] - yhat_c).mean()

        # Pooled Regression - regress Y ~ X + A
        x_t = covmatch[t].assign(treatment=t).set_index("sample_id")
        x_s = covmatch[s].assign(treatment=s).set_index("sample_id")
        x_p = x_s.append(x_t)

        y_t = covmatch.outcomes[t]
        y_t.index = pd.to_numeric(covmatch[t].sample_id, downcast="integer")
        y_s = covmatch.outcomes[s]
        y_s.index = pd.to_numeric(covmatch[s].sample_id, downcast="integer")
        y_p = y_s.append(y_t)
        fit_p = sm.OLS(y_p, sm.add_constant(x_p)).fit()
        params_p = fit_p.params
        yhat_p = covmatch.outcomes[t] - covmatch.delta @ params_p[1:-1]
        tau_p = (covmatch.outcomes[s] - yhat_p).mean()

        regression_outcomes = {
            "difference": dict(
                x=x_d, y=y_d, fit=fit_d, params=params_d, yhat=yhat_d, tau=tau_d
            ),
            "control": dict(
                x=x_c, y=y_c, fit=fit_c, params=params_c, yhat=yhat_c, tau=tau_c
            ),
            "pooled": dict(
                x=x_p, y=y_p, fit=fit_p, params=params_p, yhat=yhat_p, tau=tau_p
            ),
        }
        return regression_outcomes

In [2]:


class OneDimensionalNearestNeighbors(object):
    """Simple one dimensional nearest neighbor object


    Attributes:
        metric (str) : Scalar distance metric. Either "logit" or
           "euclidean". Logit is recommended for comparing 
           probabibilities, such as propensity scores.

    """

    def __init__(self, metric="logit"):
        self.metric = metric

    def fit(self, X):
        if len(X.shape) > 1 and X.shape[1] > 1:
            raise ValueError("This object only works with 1d data.")
        self.index = np.array(sorted([(v, idx) for idx, v in enumerate(X)]))
        return self

    def kneighbors(self, X, n_neighbors=1):
        """Find nearest neighbors

        Args:
            X (np.array): Array of samples to check of shape (N,) or (N,1).
            n_neighbors (int, optional): Number of neighbors to search for.
                Defaults to 1.

        Returns:
            (distances, indices): Two np.array objects of shape (N,n_neighbors)
                containing the distances and indices of the closest neighbors.
        """
        import bisect

        def single_sample_kneighbor(x):
            insert_idx = bisect.bisect(self.index[:, 0], x)
            lo = max(insert_idx - n_neighbors, 0)
            hi = min(insert_idx + n_neighbors, len(self.index))
            distances = [(self.dist(x, val), index)
                         for val, index in self.index[lo:hi]]
            distances, indices = list(zip(*sorted(distances)[:n_neighbors]))
            return distances, indices
        distances, indices = zip(*[single_sample_kneighbor(x) for x in X])
        #indices = np.array(indices).astype('int64')
        return np.array(distances), np.array(indices)

    def dist(self, i, j):
        """Distance between two scalars

        Args:
            i (float): first sample
            j (float): second sample

        Raises:
            NotImplementedError: Raised when non-supported metric is requested

        Returns:
            distance (float): distance between samples
        """
        if self.metric == "logit":
            return abs(np.log(i/(1-i)) - np.log(j/(1-j)))
        elif self.metric == "euclidean":
            return abs(i - j)
        else:
            raise NotImplementedError("Metric not supported")

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            if parameter == "metric_params":
                self.__dict__.update(value)
            else:
                setattr(self, parameter, value)
        return self
